In [5]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
df = pd.read_csv('df_final_eda.csv')

df_ml = pd.DataFrame(df)

In [3]:
df_ml.head()

,Unnamed: 0,user_id_x,Item_id,Item_name,Hours_Played,genres,app_name,release_year,id,user_id_y,Recommend,Year_Posted,sentiment_analysis
0,0,76561198035860222,65930,The Bureau: XCOM Declassified,0.00,Action,The Bureau: XCOM Declassified,2013,65930,longinus,False,2014,0
1,1,76561198035860222,65930,The Bureau: XCOM Declassified,0.00,Action,The Bureau: XCOM Declassified,2013,65930,manuel_vega182002,True,2015,2
2,2,GGargula,65930,The Bureau: XCOM Declassified,13.57,Action,The Bureau: XCOM Declassified,2013,65930,longinus,False,2014,0
3,3,GGargula,65930,The Bureau: XCOM Declassified,13.57,Action,The Bureau: XCOM Declassified,2013,65930,manuel_vega182002,True,2015,2
4,4,76561198085547365,65930,The Bureau: XCOM Declassified,0.00,Action,The Bureau: XCOM Declassified,2013,65930,longinus,False,2014,0


In [4]:
df_final_ml = df_ml[['genres', 'app_name','Item_name','Hours_Played', 'release_year','sentiment_analysis']]

In [5]:
df_final_ml.shape

(14939582, 6)

In [9]:
import random

# Establecemos la semilla para el generador de números aleatorios
random.seed(0)

# Supongamos que 'merged_df1_eda' es tu dataframe
# merged_df1_eda = ...

# Calculamos el 25% del total de registros
n = int(len(df_final_ml) * 0.01)

# Seleccionamos una muestra aleatoria del 25% de los registros
df_final_ml = df_final_ml.sample(n)

# Ahora 'sample_df' es tu nuevo dataframe con el 25% de los registros de 'merged_df1_eda'


In [10]:
df_final_ml.head()

,genres,app_name,Item_name,Hours_Played,release_year,sentiment_analysis
1161488,Action,Counter-Strike: Global Offensive,Counter-Strike: Global Offensive,5.68,2012,0
8930902,Indie,Loadout,Loadout,0.00,2014,2
13437565,Adventure,Emily is Away,Emily is Away,1.05,2015,2
6785745,Action,Left 4 Dead 2,Left 4 Dead 2,0.00,2009,2
8659758,Indie,Contagion,Contagion,1.22,2013,0


In [25]:


from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Definir las columnas numéricas y categóricas
num_cols = ['Hours_Played', 'release_year', 'sentiment_analysis']
cat_cols = ['genres', 'app_name']

# Crear un transformador de columnas para procesar las columnas numéricas y categóricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', num_cols),
        ('cat', OneHotEncoder(), cat_cols)])

# Seleccionar una muestra aleatoria de 10000 ítems y resetear los índices
sample_df = df_final_ml.sample(10000).reset_index(drop=True)

# Ajustar y transformar los datos con el preprocesador
X_transformed = preprocessor.fit_transform(sample_df)

# Calcular la matriz de similitud del coseno
similarity_matrix = cosine_similarity(X_transformed)







In [8]:
df1 = pd.read_csv('/home/alfredo/Documentos/Proyecto-Individual-MLOps/Datasets/funcion_1.csv')

In [9]:
df1.head()

,Unnamed: 0,genres,release_year,Hours_Played
0,0,Action,1983,59.74
1,1,Action,1984,6.40
2,2,Action,1988,270.71
3,3,Action,1989,10.11
4,4,Action,1990,313.17


In [28]:
def recomendacion_juego(item_name):
    # Verificar si el juego ingresado está en los datos
    if item_name not in sample_df['Item_name'].values:
        return "El juego no se encontró en los datos solicitados"
    
    # Obtener el índice del juego en la muestra
    item_id = sample_df[sample_df['Item_name'] == item_name].index[0]
    
    # Obtener las puntuaciones de similitud para el ítem dado
    similarity_scores = list(enumerate(similarity_matrix[item_id]))
    
    # Ordenar las puntuaciones de similitud en orden descendente
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    
    # Obtener los índices de los juegos más similares
    similar_items = [i[0] for i in similarity_scores if sample_df['Item_name'].iloc[i[0]] != item_name]
    
    # Devolver los nombres de los 5 juegos más similares sin duplicados
    recommended_games = []
    for item in similar_items:
        game_name = sample_df['Item_name'].iloc[item]
        if game_name not in recommended_games:
            recommended_games.append(game_name)
        if len(recommended_games) == 5:
            break
    
    return "Los 5 juegos más recomendados para el juego {} son: {}".format(item_name, recommended_games)

